In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@2c395174

In [2]:
import spark.implicits._

case class Movie(movie_id: Int, title: String)
case class User(user_id: Int, name: String)
case class MovieRating(movie_id: Int, user_id: Int, rating: Int, created_at: java.sql.Date)

val moviesData: Seq[(Int, String)] = Seq(
  (1, "Avengers"),
  (2, "Frozen 2"),
  (3, "Joker")
)

val usersData: Seq[(Int, String)] = Seq(
  (1, "Daniel"),
  (2, "Monica"),
  (3, "Maria"),
  (4, "James")
)

val movieRatingData: Seq[(Int, Int, Int, String)] = Seq(
  (1, 1, 3, "2020-01-12"),
  (1, 2, 4, "2020-02-11"),
  (1, 3, 2, "2020-02-12"),
  (1, 4, 1, "2020-01-01"),
  (2, 1, 5, "2020-02-17"),
  (2, 2, 2, "2020-02-01"),
  (2, 3, 2, "2020-03-01"),
  (3, 1, 3, "2020-02-22"),
  (3, 2, 4, "2020-02-25")
)

val moviesDS = moviesData.toDF("movie_id", "title").as[Movie]
val usersDS = usersData.toDF("user_id", "name").as[User]
val movieRatingDS = movieRatingData.toDF("movie_id", "user_id", "rating", "created_at")
  .withColumn("created_at", col("created_at").cast("date"))
  .as[MovieRating]


import spark.implicits._


defined class Movie
defined class User
defined class MovieRating
moviesData: Seq[(Int, String)] = List(
  (1, "Avengers"),
  (2, "Frozen 2"),
  (3, "Joker")
)
usersData: Seq[(Int, String)] = List(
  (1, "Daniel"),
  (2, "Monica"),
  (3, "Maria"),
  (4, "James")
)
movieRatingData: Seq[(Int, Int, Int, String)] = List(
  (1, 1, 3, "2020-01-12"),
  (1, 2, 4, "2020-02-11"),
  (1, 3, 2, "2020-02-12"),
  (1, 4, 1, "2020-01-01"),
  (2, 1, 5, "2020-02-17"),
  (2, 2, 2, "2020-02-01"),
  (2, 3, 2, "2020-03-01"),
  (3, 1, 3, "2020-02-22"),
  (3, 2, 4, "2020-02-25")
)
moviesDS: Dataset[Movie] = [movie_id: int, title: string]
usersDS: Dataset[User] = [user_id: int, name: string]
movieRatingDS: Dataset[MovieRating] = [movie_id: int, user_id: int ... 2 more fields]

Write a solution to:

* Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
* Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.

The result format is in the following example.

| results      |
|--------------|
| Daniel       |
| Frozen 2     |

In [3]:
movieRatingDS.show()
val windowSpec = Window.orderBy(col("cnt").desc)
val topUserDF = movieRatingDS
    .groupBy(col("user_id"))
    .agg(count(lit(1)).as("cnt"))
    .withColumn("rank", dense_rank().over(windowSpec))
    .filter(col("rank") === 1)
    .select(col("user_id"))
    .join(usersDS, Seq("user_id"), "left_outer")
    .sort(col("name").asc)
    .select(col("name").as("results"))
    .limit(1)

val windowSpec2 = Window.orderBy(col("avg").desc)

val topMovieDF = movieRatingDS
    .filter(date_trunc("month", col("created_at")) === to_date(lit("2020-02-01")))
    .groupBy(col("movie_id"))
    .agg(avg(col("rating")).as("avg"))
    .withColumn("rank", dense_rank().over(windowSpec2))
    .filter(col("rank") === 1)
    .select(col("movie_id"))
    .join(moviesDS, Seq("movie_id"), "left_outer")
    .sort(col("title").asc)
    .select(col("title").as("results"))
    .limit(1)

topUserDF.union(topMovieDF).show()

+--------+-------+------+----------+
|movie_id|user_id|rating|created_at|
+--------+-------+------+----------+
|       1|      1|     3|2020-01-12|
|       1|      2|     4|2020-02-11|
|       1|      3|     2|2020-02-12|
|       1|      4|     1|2020-01-01|
|       2|      1|     5|2020-02-17|
|       2|      2|     2|2020-02-01|
|       2|      3|     2|2020-03-01|
|       3|      1|     3|2020-02-22|
|       3|      2|     4|2020-02-25|
+--------+-------+------+----------+

+--------+
| results|
+--------+
|  Daniel|
|Frozen 2|
+--------+



windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@eae4923
topUserDF: Dataset[Row] = [results: string]
windowSpec2: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@80d5523
topMovieDF: Dataset[Row] = [results: string]

In [4]:
spark.stop()